In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import kurtosis, iqr, skew
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.set_option('display.max_columns', 500)



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tr_data = pd.read_csv('/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/Data/Data/train_Data.csv')
tr_bear = pd.read_csv('/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/Data/Data/train_bureau.csv')

ts_data = pd.read_csv('/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/Data/Data/test_Data.csv')
ts_bear = pd.read_csv('/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/Data/Data/test_bureau.csv')

s = pd.read_csv('/kaggle/input/analytics-vidhya-ltfs-data-science-finhack-3/sample_submission_ejm25Dc.csv')

In [ ]:
def reduce_memory(df):
    """Reduce memory usage of a dataframe by setting data types. """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Initial df memory usage is {:.2f} MB for {} columns'
          .format(start_mem, len(df.columns)))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                # Can use unsigned int here too
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    memory_reduction = 100 * (start_mem - end_mem) / start_mem
    print('Final memory usage is: {:.2f} MB - decreased by {:.1f}%'.format(end_mem, memory_reduction))
    return df

In [ ]:
tr_data, tr_bear = reduce_memory(tr_data), reduce_memory(tr_bear)
ts_data, ts_bear = reduce_memory(ts_data), reduce_memory(ts_bear)

In [ ]:
tr_data['LoanStatus'].value_counts()


tr_data.groupby(['LoanStatus','Top-up Month']).count()

In [ ]:
tr_data['LoanStatus'].value_counts()

In [ ]:
tr_data['Top-up Month'].value_counts()

In [ ]:
# tr_data['BranchID'].unique()

# print(tr_data.shape,ts_data.shape)
# from sklearn.utils import shuffle

# tr_dataa = tr_data.copy()

# tr_data = tr_dataa[tr_dataa['Top-up Month']!='No Top-up Service'].copy()

# tr_dataa = tr_dataa[tr_dataa['Top-up Month']=='No Top-up Service'].sample(50000,random_state=1994)

# tr_data= tr_data.append(tr_dataa,ignore_index=False)

# tr_data = shuffle(tr_data)

In [ ]:
# tr_bear
tr_data.shape,ts_data.shape

In [ ]:
df=tr_data.append(ts_data,ignore_index=True)
df.shape

In [ ]:
df['Top-up Month'].unique()

label_map={' > 48 Months':0, 'No Top-up Service':1, '12-18 Months':2,
       '36-48 Months':3, '18-24 Months':4, '24-30 Months':5, '30-36 Months':6}

inverted_label_map = {value: key for key, value in label_map.items()}

print(inverted_label_map)

df['Top-up Month']=df['Top-up Month'].map(label_map)

In [ ]:
bear = tr_bear.append(ts_bear,ignore_index=True)
bear

In [ ]:
bear = bear.merge(df[['ID','DisbursalDate']],on='ID',how='left')
bear.head()

In [ ]:
# bear.merge(bear[['ID','DisbursalDate','ACCT-TYPE']],on=['ID','DisbursalDate'],how='left')
bear['DISBURSED-DT']=pd.to_datetime(bear['DISBURSED-DT'])
bear['DisbursalDate']=pd.to_datetime(bear['DisbursalDate'])
import gc
gc.collect()

In [ ]:


# bear.groupby('ID').filter(lambda x: any((x['SELF-INDICATOR']==True) & (x['DISBURSED-DT']>=x['DisbursalDate'])))
# bear.groupby('ID').apply('size')


In [ ]:
bear = bear[bear['DisbursalDate']<=bear['DISBURSED-DT']].drop('DisbursalDate',axis=1).copy()

In [ ]:
# bear['acc_type']=np.where((bear['DISBURSED-DT']>=bear['DisbursalDate']) & (bear['SELF-INDICATOR']==True),'1','0')
# bear = bear[bear['acc_type']=='1'].drop(['DisbursalDate'],axis=1).copy()
# bear.shape



In [ ]:
bear.shape

In [ ]:
print(tr_data['Top-up Month'].unique())
tr_data[tr_data['Top-up Month']==' > 48 Months']['ID'].values[:10]

In [ ]:
bear[bear['ID']==7].sort_values(by='DISBURSED-DT')

In [ ]:
tr_data[tr_data['ID']==7]

In [ ]:
bear.shape

In [ ]:
bear['disbursal_date_diff']=bear.sort_values(['ID','DISBURSED-DT']).groupby('ID')['DISBURSED-DT'].diff()/np.timedelta64(1,'M')

In [ ]:
bear['disbursal_date_from_min']=bear.groupby('ID')['DISBURSED-DT'].transform(lambda x: (x-x.min())/np.timedelta64(1,'M'))
bear['disbursal_date_from_min']

In [ ]:
pd.set_option('display.max_rows', 200)


# tr_data.groupby('ZiPCODE')['Top-up Month'].unique().reset_index()

In [ ]:
bear['DATE-REPORTED']=pd.to_datetime(bear['DATE-REPORTED'],errors='coerce')
# bear['DISBURSED-DT']=pd.to_datetime(bear['DISBURSED-DT'],errors='coerce')
bear['CLOSE-DT']=pd.to_datetime(bear['CLOSE-DT'],errors='coerce')

bear['LAST-PAYMENT-DATE']=pd.to_datetime(bear['LAST-PAYMENT-DATE'],errors='coerce')

In [ ]:
bear[bear['ID']==5].sort_values(by='DISBURSED-DT')

In [ ]:
# bear.groupby('ID')['DATE-REPORTED'].min()

In [ ]:
# bear.groupby('ID')['DATE-REPORTED'].max().value_counts()
# 

In [ ]:
def clean_install_freq(val):
    v = str(val).split('/')
    if len(v)==1:
        return np.nan
    else:
        return str(v[1]).replace(',', '')

bear['INSTALLMENT-AMT-FREQUENCY'] = bear['INSTALLMENT-AMT'].apply(clean_install_freq)

In [ ]:
# bear['INSTALLMENT-AMT']

In [ ]:
bear['dd_DATE-REPORTED_y']=bear['DATE-REPORTED'].dt.year
bear['dd_DATE-REPORTED_m']=bear['DATE-REPORTED'].dt.month
bear['dd_DATE-REPORTED_d']=bear['DATE-REPORTED'].dt.day
bear['dd_DATE-REPORTED_w']=bear['DATE-REPORTED'].dt.week
bear['dd_DATE-REPORTED_dayofweek']=bear['DATE-REPORTED'].dt.weekday
bear['dayofyear_dd_DATE-REPORTED'] = bear['DATE-REPORTED'].dt.dayofyear
bear['weekofyear_dd_DATE-REPORTED'] = bear['DATE-REPORTED'].dt.weekofyear
bear['is_month_start_dd_DATE-REPORTED'] = (bear['DATE-REPORTED'].dt.is_month_start).astype(int)
bear['is_month_end_dd_DATE-REPORTED'] = (bear['DATE-REPORTED'].dt.is_month_end).astype(int)
bear['is_weekend_dd_DATE-REPORTED'] = bear['DATE-REPORTED'].apply(lambda x: 1 if x.weekday() in (5, 6) else 0)
bear['last_month_of_year_dd_DATE-REPORTED'] = bear['DATE-REPORTED'].apply(lambda x: 1 if x.month == 12 else 0)
bear['Quarter_dd_DATE-REPORTED'] = bear['DATE-REPORTED'].dt.quarter
bear['QSQuarter_dd_DATE-REPORTED'] = (bear['DATE-REPORTED'].dt.is_quarter_start).astype(int)
bear['QEQuarter_dd_DATE-REPORTED'] = (bear['DATE-REPORTED'].dt.is_quarter_end).astype(int)


# bear['dd_DISBURSED-DT_y']=bear['DISBURSED-DT'].dt.year
# bear['dd_DISBURSED-DT_m']=bear['DISBURSED-DT'].dt.month
# bear['dd_DISBURSED-DT_d']=bear['DISBURSED-DT'].dt.day
# bear['dd_DISBURSED-DT_w']=bear['DISBURSED-DT'].dt.week
# bear['dd_DISBURSED-DT_dayofweek']=bear['DISBURSED-DT'].dt.weekday


# bear['dd_CLOSE-DT_y']=bear['CLOSE-DT'].dt.year
# bear['dd_CLOSE-DT_m']=bear['CLOSE-DT'].dt.month
# bear['dd_CLOSE-DT_d']=bear['CLOSE-DT'].dt.day
# bear['dd_CLOSE-DT_w']=bear['CLOSE-DT'].dt.week
# bear['dd_CLOSE-DT_dayofweek']=bear['CLOSE-DT'].dt.weekday


# bear['dd_LAST-PAYMENT-DATE_y']=bear['LAST-PAYMENT-DATE'].dt.year
# bear['dd_LAST-PAYMENT-DATE_m']=bear['LAST-PAYMENT-DATE'].dt.month
# bear['dd_LAST-PAYMENT-DATE_d']=bear['LAST-PAYMENT-DATE'].dt.day
# bear['dd_LAST-PAYMENT-DATE_w']=bear['LAST-PAYMENT-DATE'].dt.week
# bear['dd_LAST-PAYMENT-DATE_dayofweek']=bear['LAST-PAYMENT-DATE'].dt.weekday






bear['date_diff_1'] = (bear['DATE-REPORTED'] - bear['DISBURSED-DT'])/np.timedelta64(1,'M')
bear['date_diff_2']=(bear['CLOSE-DT'] - bear['DISBURSED-DT'])/np.timedelta64(1,'M')
bear['date_diff_3']=(bear['CLOSE-DT'] - bear['LAST-PAYMENT-DATE'])/np.timedelta64(1,'M')
bear['date_diff_4']=(bear['LAST-PAYMENT-DATE'] - bear['DISBURSED-DT'])/np.timedelta64(1,'M')

In [ ]:
# bear['date_diff_4']

In [ ]:
bear['CURRENT-BAL'] = bear['CURRENT-BAL'].apply(lambda x: str(x).replace(',', '')).astype(np.float64)
bear['OVERDUE-AMT'] = bear['OVERDUE-AMT'].apply(lambda x: str(x).replace(',', '')).astype(np.float64)
bear['CREDIT-LIMIT/SANC AMT'] = bear['CREDIT-LIMIT/SANC AMT'].apply(lambda x: str(x).replace(',', '')).astype(np.float64)
bear['DISBURSED-AMT/HIGH CREDIT'] = bear['DISBURSED-AMT/HIGH CREDIT'].apply(lambda x: str(x).replace(',', '')).astype(np.float64)


bear['HIGH-CURRENT-DIFF']  = bear['DISBURSED-AMT/HIGH CREDIT']-bear['CURRENT-BAL']

def clean_install_amt(val):
    v = str(val).split('/')
    return float(str(v[0]).replace(',', ''))

bear['INSTALLMENT-AMT']=bear['INSTALLMENT-AMT'].apply(clean_install_amt)

def history_data(val):

    v = str(val).split(",")
    v = [float(x) for x in v if x not in ['','0','nan']]
    if len(v)==0:
        return 0
    else:
        return sum(v)/len(v)
    
def history_count(val):

    v = str(val).split(",")
    v = [float(x) for x in v if x not in ['','0','nan']]
    if len(v)==0:
        return 0
    else:
        return len(v)

bear['AMT OVERDUE - HIST'] = bear['AMT OVERDUE - HIST'].apply(history_data)

bear['AMT PAID - HIST'] = bear['AMT PAID - HIST'].apply(history_data)
bear['CUR BAL - HIST'] = bear['CUR BAL - HIST'].apply(history_data)



bear['REPORTED DATE - HIST'] = bear['REPORTED DATE - HIST'].apply(history_count)

In [ ]:
bear['F1'] = bear.sort_values(by=['ID','DISBURSED-DT']).groupby('ID')[['DISBURSED-AMT/HIGH CREDIT']].cumsum().sort_index()
bear['F2'] = bear.sort_values(by=['ID','DISBURSED-DT']).groupby('ID')[['INSTALLMENT-AMT']].cumsum().sort_index()
bear['F3'] = bear.sort_values(by=['ID','DISBURSED-DT']).groupby('ID')[['CURRENT-BAL']].cumsum().sort_index()
bear['F4'] = bear.sort_values(by=['ID','DISBURSED-DT']).groupby('ID')[['OVERDUE-AMT']].cumsum().sort_index()


bear['F5'] = bear.sort_values(by=['ID','DATE-REPORTED']).groupby('ID')[['DISBURSED-AMT/HIGH CREDIT']].cumsum().sort_index()
bear['F6'] = bear.sort_values(by=['ID','DATE-REPORTED']).groupby('ID')[['INSTALLMENT-AMT']].cumsum().sort_index()
bear['F7'] = bear.sort_values(by=['ID','DATE-REPORTED']).groupby('ID')[['CURRENT-BAL']].cumsum().sort_index()
bear['F8'] = bear.sort_values(by=['ID','DATE-REPORTED']).groupby('ID')[['OVERDUE-AMT']].cumsum().sort_index()

bear['F9'] = bear.sort_values(by=['ID','CLOSE-DT']).groupby('ID')[['DISBURSED-AMT/HIGH CREDIT']].cumsum().sort_index()
bear['F10'] = bear.sort_values(by=['ID','CLOSE-DT']).groupby('ID')[['INSTALLMENT-AMT']].cumsum().sort_index()
bear['F11'] = bear.sort_values(by=['ID','CLOSE-DT']).groupby('ID')[['CURRENT-BAL']].cumsum().sort_index()
bear['F12'] = bear.sort_values(by=['ID','CLOSE-DT']).groupby('ID')[['OVERDUE-AMT']].cumsum().sort_index()


bear['F13'] = bear.sort_values(by=['ID','LAST-PAYMENT-DATE']).groupby('ID')[['DISBURSED-AMT/HIGH CREDIT']].cumsum().sort_index()
bear['F14'] = bear.sort_values(by=['ID','LAST-PAYMENT-DATE']).groupby('ID')[['INSTALLMENT-AMT']].cumsum().sort_index()
bear['F11'] = bear.sort_values(by=['ID','LAST-PAYMENT-DATE']).groupby('ID')[['CURRENT-BAL']].cumsum().sort_index()
bear['F12'] = bear.sort_values(by=['ID','LAST-PAYMENT-DATE']).groupby('ID')[['OVERDUE-AMT']].cumsum().sort_index()

In [ ]:
bear.head()

In [ ]:
bear['ACCOUNT-STATUS-CHANGE']=bear.sort_values(['ID','DISBURSED-DT'])['ACCOUNT-STATUS'].shift(1, fill_value='other') != bear.sort_values(['ID','DISBURSED-DT'])['ACCOUNT-STATUS']

bear['SELF-INDICATOR-CHANGE']=bear.sort_values(['ID','DISBURSED-DT'])['SELF-INDICATOR'].shift(1, fill_value='other') != bear.sort_values(['ID','DISBURSED-DT'])['SELF-INDICATOR']

bear['ACCT-TYPE-CHANGE']=bear.sort_values(['ID','DISBURSED-DT'])['ACCT-TYPE'].shift(1, fill_value='other') != bear.sort_values(['ID','DISBURSED-DT'])['ACCT-TYPE']

bear['OWNERSHIP-IND-CHANGE']=bear.sort_values(['ID','DISBURSED-DT'])['OWNERSHIP-IND'].shift(1, fill_value='other') != bear.sort_values(['ID','DISBURSED-DT'])['OWNERSHIP-IND']


bear['CONTRIBUTOR-TYPE-CHANGE']=bear.sort_values(['ID','DISBURSED-DT'])['CONTRIBUTOR-TYPE'].shift(1, fill_value='other') != bear.sort_values(['ID','DISBURSED-DT'])['CONTRIBUTOR-TYPE']

In [ ]:
bear.head()

In [ ]:
bear['DPD - HIST_Zero'] = bear['DPD - HIST'].str.count('000')
bear['DPD - HIST_D'] = bear['DPD - HIST'].str.count('DDD')
bear['DPD - HIST_X'] = bear['DPD - HIST'].str.count('XXX')
bear['DPD - HIST_len'] = bear['DPD - HIST'].str.len()
bear['DPD - HIST_NUM'] = bear['DPD - HIST'].str.count('\d{3}')

In [ ]:
# bear['DPD - HIST_D']

In [ ]:
# bear['DPD - HIST_NUM']

In [ ]:
# agg_bear = {
#     'SELF-INDICATOR' : ['sum'],
#     'MATCH-TYPE':['count','nunique'],
#     'ACCT-TYPE':['count','nunique'],
#     'CONTRIBUTOR-TYPE':['count','nunique'],
#     'OWNERSHIP-IND':['count','nunique'],
#     'ACCOUNT-STATUS':['count','nunique'],
#     'ASSET_CLASS':['count','nunique'],
#     'INSTALLMENT-FREQUENCY':['count','nunique'],
# #     'TENURE':['sum','max','min','mean']
# #     'TENURE':['sum']
# }

# agg_bear = {
#     'SELF-INDICATOR' : ['sum'],
#     'MATCH-TYPE':['count','nunique'],
#     'ACCT-TYPE':['count','nunique'],
#     'CONTRIBUTOR-TYPE':['count','nunique'],
#     'OWNERSHIP-IND':['count','nunique'],
#     'ACCOUNT-STATUS':['count','nunique'],
#     'ASSET_CLASS':['count','nunique'],
#     'INSTALLMENT-FREQUENCY':['count','nunique'],
#     'CURRENT-BAL':['sum','max','min','mean'],
#     'TENURE':['sum','max','min','mean'],
#     'OVERDUE-AMT':['sum','max','min','mean'],
#     'CREDIT-LIMIT/SANC AMT':['sum','max','min','mean'],
#     'DISBURSED-AMT/HIGH CREDIT':['sum','max','min','mean'],
#     'INSTALLMENT-AMT':['sum','max','min','mean'],
#     'AMT OVERDUE - HIST':['mean'],
#     'AMT PAID - HIST':['mean'],
#     'CUR BAL - HIST':['mean'],
#     'REPORTED DATE - HIST':['mean','max']
# }

agg_bear = {
    'SELF-INDICATOR' : ['sum','mean'],
    'MATCH-TYPE':['size','nunique'],
    'ACCT-TYPE':['nunique'],
    'CONTRIBUTOR-TYPE':['nunique'],
    'OWNERSHIP-IND':['nunique'],
    'ACCOUNT-STATUS':['nunique'],
    'ASSET_CLASS':['size','nunique'],
    'INSTALLMENT-FREQUENCY':['size','nunique'],
    'TENURE':['sum','max','min','mean'],
    #39.7
    'CURRENT-BAL':['sum','max','min','mean'],
    'OVERDUE-AMT':['sum','max','min','mean'],
    
    'CREDIT-LIMIT/SANC AMT':['sum','max','min','mean','quantile'],
    'DISBURSED-AMT/HIGH CREDIT':['sum','max','min','mean','quantile'],
    'INSTALLMENT-AMT':['sum','max','min','mean','quantile'],
    'WRITE-OFF-AMT':['sum','max','min','mean','quantile'],
    
    'AMT OVERDUE - HIST':['mean','max','min'],
    'AMT PAID - HIST':['mean','max','min'],
    'CUR BAL - HIST':['mean','max','min'],
    'REPORTED DATE - HIST':['mean','max','min'],
    'date_diff_1':['mean','max','min','sum'],
    'date_diff_2':['mean','max','min','sum'],
    'date_diff_3':['mean','max','min','sum'],
    'date_diff_4':['mean','max','min','sum'],
    
    
    'INSTALLMENT-AMT-FREQUENCY':['size','nunique'],
    'HIGH-CURRENT-DIFF':['sum','max','min','mean','quantile'],
    
    
    'disbursal_date_diff':['sum','max','min','mean'],
    'disbursal_date_from_min':['sum','max','min','mean'],
    'ACCOUNT-STATUS-CHANGE': ['sum','mean'],
    'SELF-INDICATOR-CHANGE': ['sum','mean'],
    'ACCT-TYPE-CHANGE': ['sum','mean'],
    'CONTRIBUTOR-TYPE-CHANGE': ['sum','mean'],
    'OWNERSHIP-IND-CHANGE': ['sum','mean'],
    'DPD - HIST_Zero' : ['sum','mean'],
    'DPD - HIST_D' : ['sum','mean'],
    'DPD - HIST_X' : ['sum','mean'],
    'DPD - HIST_len' : ['sum','mean'],
    'DPD - HIST_NUM': ['sum','mean']
}

for c in bear.columns:
    if c.startswith("dd_"):
        agg_bear[c]=['count','nunique']
    elif c.startswith("F"):
        agg_bear[c]=['mean','max','min']

        
df_bear = bear.groupby(['ID']).agg(agg_bear)

df_bear.columns=['A_' + '_'.join(col).strip() for col in df_bear.columns.values]
df_bear.reset_index(inplace=True)
df_bear.head()

In [ ]:
df_bear.fillna(0,inplace=True)

In [ ]:
df=df.merge(df_bear,on='ID',how='left')
df.shape

In [ ]:
df

In [ ]:
df['SupplierID'].nunique()

In [ ]:
df.isnull().sum()

In [ ]:
df.head(3)

In [ ]:
df['DisbursalDate']=pd.to_datetime(df['DisbursalDate'])

df['DisbursalDate_y']=df['DisbursalDate'].dt.year
df['DisbursalDate_m']=df['DisbursalDate'].dt.month
df['DisbursalDate_d']=df['DisbursalDate'].dt.day
df['DisbursalDate_w']=df['DisbursalDate'].dt.week
df['DisbursalDatedayofweek']=df['DisbursalDate'].dt.weekday
df['DisbursalDatedayofyear'] = df['DisbursalDate'].dt.dayofyear
df['DisbursalDateweekofyear'] = df['DisbursalDate'].dt.weekofyear
df['DisbursalDateis_month_start'] = (df['DisbursalDate'].dt.is_month_start).astype(int)
df['DisbursalDateis_month_end'] = (df['DisbursalDate'].dt.is_month_end).astype(int)
df['DisbursalDateis_weekend'] = df['DisbursalDate'].apply(lambda x: 1 if x.weekday() in (5, 6) else 0)
df['DisbursalDatelast_month_of_year'] = df['DisbursalDate'].apply(lambda x: 1 if x.month == 12 else 0)
df['DisbursalDateQuarter'] = df['DisbursalDate'].dt.quarter
df['DisbursalDateQSQuarter'] = (df['DisbursalDate'].dt.is_quarter_start).astype(int)
df['DisbursalDateQEQuarter'] = (df['DisbursalDate'].dt.is_quarter_end).astype(int)


df['MaturityDAte']=pd.to_datetime(df['MaturityDAte'])

df['MaturityDAte_y']=df['MaturityDAte'].dt.year
df['MaturityDAte_m']=df['MaturityDAte'].dt.month
df['MaturityDAte_d']=df['MaturityDAte'].dt.day
df['MaturityDAte_w']=df['MaturityDAte'].dt.week
df['MaturityDAtedayofweek']=df['MaturityDAte'].dt.weekday
df['dayofyear_MD'] = df['MaturityDAte'].dt.dayofyear
df['weekofyear_MD'] = df['MaturityDAte'].dt.weekofyear
df['is_month_start_MD'] = (df['MaturityDAte'].dt.is_month_start).astype(int)
df['is_month_end_MD'] = (df['MaturityDAte'].dt.is_month_end).astype(int)
df['is_weekend_MD'] = df['MaturityDAte'].apply(lambda x: 1 if x.weekday() in (5, 6) else 0)
df['last_month_of_year_MD'] = df['MaturityDAte'].apply(lambda x: 1 if x.month == 12 else 0)
df['Quarter_MD'] = df['MaturityDAte'].dt.quarter
df['QSQuarter_MD'] = (df['MaturityDAte'].dt.is_quarter_start).astype(int)
df['QEQuarter_MD'] = (df['MaturityDAte'].dt.is_quarter_end).astype(int)


df['AuthDate']=pd.to_datetime(df['AuthDate'])

df['AuthDate_y']=df['AuthDate'].dt.year
df['AuthDate_m']=df['AuthDate'].dt.month
df['AuthDate_d']=df['AuthDate'].dt.day
df['AuthDate_w']=df['AuthDate'].dt.week
df['AuthDatedayofweek']=df['AuthDate'].dt.weekday
df['dayofyear_AD'] = df['AuthDate'].dt.dayofyear
df['weekofyear_AD'] = df['AuthDate'].dt.weekofyear
df['is_month_start_AD'] = (df['AuthDate'].dt.is_month_start).astype(int)
df['is_month_end_AD'] = (df['AuthDate'].dt.is_month_end).astype(int)
df['is_weekend_AD'] = df['AuthDate'].apply(lambda x: 1 if x.weekday() in (5, 6) else 0)
df['last_month_of_year_AD'] = df['AuthDate'].apply(lambda x: 1 if x.month == 12 else 0)
df['Quarter_AD'] = df['AuthDate'].dt.quarter
df['QSQuarter_AD'] = (df['AuthDate'].dt.is_quarter_start).astype(int)
df['QEQuarter_AD'] = (df['AuthDate'].dt.is_quarter_end).astype(int)


df['diff_1'] = (df['MaturityDAte'] - df['DisbursalDate'])/np.timedelta64(1,'M')
df['diff_2']=(df['AuthDate'] - df['DisbursalDate'])/np.timedelta64(1,'D')
df['diff_3']=(df['MaturityDAte'] - df['AuthDate'])/np.timedelta64(1,'D')

cat=['Frequency','InstlmentMode','LoanStatus','PaymentMode','Area','AssetID','SEX','City','State']
# 'BranchID',,'ManufacturerID','SupplierID','AGE',,'ZiPCODE'

for k in cat:
    df[k]=df[k].astype('category')
    
    
df['totAmnt']=df['AmountFinance']-df['DisbursalAmount']
df['incomeleft']=df['MonthlyIncome']-df['EMI']
df['incomeleft']=df['incomeleft'].apply(lambda x: 0 if x<=0 else x)
df['Asset_left_to_pay']=df['AssetCost']-df['EMI']*df['Tenure']
df['Asset_left_to_pay']=df['Asset_left_to_pay'].apply(lambda x: 0 if x<=0 else x)
df['interest_payable']=df['EMI']*df['Tenure']-df['DisbursalAmount']
df['extra_finance']=df['DisbursalAmount'] - df['AssetCost']*(df['LTV']/100)
df['asset_disburse']=(df['AssetCost']-df['DisbursalAmount'])/df['DisbursalAmount']

df['yearly_interest'] = ((((df['EMI']*df['Tenure'])/df['DisbursalAmount'])-1)/df['Tenure'])*12*100

df['total_principal_plus_interest'] = df['EMI']*df['Tenure']
df['rem_month_tenure']=df['Tenure']- df['diff_1']
df['rem_month_tenure']=df['rem_month_tenure'].apply(lambda x: 0 if x<=0 else x)
df["loan_ratio"] = df["DisbursalAmount"] / df["AssetCost"]

df.drop(['DisbursalDate','MaturityDAte','AuthDate','AssetID'],axis=1,inplace=True)




# df.info()

In [ ]:
# pd.factorize(bear.groupby('ID')['DATE-REPORTED'].min())[0]

In [ ]:
agg_bear = {
'AssetCost': ['mean','min','max'],
'AmountFinance': ['mean','min','max'],
'Tenure' : ['mean','min','max']
}

temp = df.groupby(['ManufacturerID']).agg(agg_bear)
temp1 = df.groupby(['Area']).agg(agg_bear)

In [ ]:
df=df.merge(temp,on='ManufacturerID',how='left')
df=df.merge(temp1,on='Area',how='left')

In [ ]:

df.drop(['SupplierID', 'Area','ManufacturerID','City','State','ZiPCODE'],axis=1,inplace=True)

In [ ]:
tr_data['ManufacturerID'].value_counts().shape

In [ ]:
df['CONTRIBUTOR-TYPE'] = bear.groupby('ID')['CONTRIBUTOR-TYPE'].agg(pd.Series.mode).apply(lambda x : ''.join(x)).astype('category')
df['MATCH-TYPE'] = bear.groupby('ID')['MATCH-TYPE'].agg(pd.Series.mode).apply(lambda x : ''.join(x)).astype('category')
df['OWNERSHIP-IND'] = bear.groupby('ID')['OWNERSHIP-IND'].agg(pd.Series.mode).apply(lambda x : ''.join(x)).astype('category')
df['ACCOUNT-STATUS'] = bear.groupby('ID')['ACCOUNT-STATUS'].agg(pd.Series.mode).apply(lambda x : ''.join(x)).astype('category')
df['ACCT-TYPE'] = bear.groupby('ID')['ACCT-TYPE'].agg(pd.Series.mode).apply(lambda x : ''.join(x)).astype('category')

# df['DATE-REPORTED_MIN'] = pd.factorize(bear.groupby('ID')['DATE-REPORTED'].min())[0]
# df['DISBURSED-DT_MIN'] = pd.factorize(bear.groupby('ID')['DISBURSED-DT'].min())[0]
# df['CLOSE-DT_MIN'] = pd.factorize(bear.groupby('ID')['CLOSE-DT'].min())[0]
# df['LAST-PAYMENT-DATE_MIN'] = pd.factorize(bear.groupby('ID')['LAST-PAYMENT-DATE'].min())[0]
# DISBURSED-DT	CLOSE-DT	LAST-PAYMENT-DATE

In [ ]:
to_drop=[]
corr_matrix = df.drop(['ID','Top-up Month'],axis=1).corr()
# print(corr_matrix)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper   

to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(to_drop)

In [ ]:
df1 = df.drop(to_drop, axis=1)

In [ ]:
df1

In [ ]:
# numcols = ['Tenure', 'AssetCost', 'AmountFinance',
#        'DisbursalAmount', 'EMI','LTV','AGE', 'MonthlyIncome','diff_1', 'diff_2', 'diff_3',
#        'totAmnt', 'incomeleft', 'Asset_left_to_pay', 'interest_payable',
#        'extra_finance', 'asset_disburse']
# df1 = df.groupby(['BranchID','ZiPCODE'])[numcols].mean()

# df1.columns=['A1_' + '_'.join(col).strip() for col in df1.columns.values]
# df1.reset_index(inplace=True)


# df=df.merge(df1,on=['BranchID','ZiPCODE'],how='left')
df1.shape

In [ ]:
# df1.head()

In [ ]:
import re
df1 = df1.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', str(x)))

In [ ]:
df1 = df1.rename(columns={'TopupMonth' : 'Top-up Month'})

In [ ]:
for x in ['Frequency','InstlmentMode','LoanStatus','PaymentMode','SEX',
 'CONTRIBUTORTYPE','MATCHTYPE','OWNERSHIPIND','ACCOUNTSTATUS','ACCTTYPE']:
    df1[x] = df1[x].cat.add_categories('Unk')
    df1[x].fillna('Unk', inplace =True)

In [ ]:
dftrain=df1[df1['Top-up Month'].isnull()!=True]
dftest=df1[df1['Top-up Month'].isnull()==True]
dftest.head()

In [ ]:
dftrain['Top-up Month'].value_counts()

In [ ]:
# dftrain['ZiPCODE']=dftrain['ZiPCODE'].astype(np.float64)

from sklearn.utils import class_weight
cw = list(class_weight.compute_class_weight('balanced',
                                             np.unique(dftrain['Top-up Month']),
                                             dftrain['Top-up Month']))
cw

In [ ]:
# cw = []
# cw={0:2.1969024281957585+1,
#  1:0.17228911306360054,
#  2:17.774937828129318+4,
#  3:5.027156924038762+3,
#  4:7.761522683397684+2.5,
#  5:5.263254786450663+2,
#  6:6.0023793972193715+2.5}

# cw={0:4.1969024281957585,
#  1:3.17228911306360054,
#  2:17.774937828129318,
#  3:8.027156924038762,
#  4:10.761522683397684,
#  5:8.263254786450663,
#  6:9.0023793972193715}

In [ ]:
X,y=dftrain.drop(['Top-up Month','ID'],axis=1),dftrain['Top-up Month']
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 1994)

In [ ]:
cat_feats = ['Frequency','InstlmentMode','LoanStatus','PaymentMode','SEX',
 'CONTRIBUTORTYPE','MATCHTYPE','OWNERSHIPIND','ACCOUNTSTATUS','ACCTTYPE']

In [ ]:
X_train.head(3)

In [ ]:
from catboost import CatBoostClassifier,Pool, cv
from lightgbm import LGBMClassifier,LGBMRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,f1_score,mean_squared_error
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import f1_score
def evaluate_macroF1_lgb(truth, predictions):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

# m=LGBMClassifier(n_estimators=5000,random_state=1994,learning_rate=0.05,num_leaves=31,max_depth=-1,
#                  objective='multiclass',num_class=7,feature_fraction=0.9,bagging_fraction=0.9,
#                 reg_alpha=20,reg_lambda=10
#                 )

In [ ]:
from catboost import CatBoostClassifier
cw={0:8,
     1:3,
     2:18,
     3:8,
     4:11,
     5:8,
     6:9}
# cw={0:4.1969024281957585,
#  1:3.17228911306360054,
#  2:17.774937828129318,
#  3:8.027156924038762,
#  4:10.761522683397684,
#  5:8.263254786450663,
#  6:9.0023793972193715}
m2=CatBoostClassifier(n_estimators=1000,random_state=1994,eval_metric='TotalF1',learning_rate=0.05,
                      cat_features=cat_feats,task_type="GPU",class_weights=cw,random_strength=200,bagging_temperature=0.5,l2_leaf_reg=1)
m2.fit(X_train,y_train,eval_set=[(X_val, y_val)], early_stopping_rounds=50,verbose=200)
p=m2.predict(X_val)
print(f1_score(y_val,p,average='macro'))

In [ ]:
# max_score = 0
# best_cw = 0.1
# for x in range(1):
#     cw={0:8,
#      1:3,
#      2:18,
#      3:8,
#      4:11,
#      5:8,
#      6:9}

#     m = CatBoostClassifier(cat_features=cat_feats,task_type="GPU",n_estimators=1000,
#                            learning_rate=0.07,class_weights=cw)
#     m.fit(X_train,y_train,verbose=500)#,eval_set=[(X_train,y_train),(X_val, y_val)], early_stopping_rounds=50,verbose=200)#,eval_metric= evaluate_macroF1_lgb)
#     p=m.predict(X_val)
#     print(f1_score(y_val,p,average='macro'))
#     if f1_score(y_val,p,average='macro') > max_score:
#         max_score = f1_score(y_val,p,average='macro')
#         best_cw = cw[6]
#     cw[6] = x
#     print(cw)

In [ ]:
# best_cw

In [ ]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

sns.set_style('darkgrid')
warnings.filterwarnings('ignore')


%matplotlib inline
feature_imp = pd.DataFrame(sorted(zip(m2.feature_importances_, X_train.columns), reverse=True)[:200], 
                           columns=['Value','Feature'])
plt.figure(figsize=(12,8))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()

In [ ]:
feature_imp.shape

In [ ]:
feat_cols = feature_imp['Feature'][:200].values

In [ ]:
Xtest = dftest.drop(['Top-up Month','ID'],axis=1)

In [ ]:
y_pred1 = m2.predict(Xtest)#,num_iteration=m.best_iteration_)
dftest['Top-up Month']=y_pred1
dftest['Top-up Month']=dftest['Top-up Month'].map(inverted_label_map)
dftest[['ID','Top-up Month']]

dftest['Top-up Month'].value_counts()

In [ ]:
# X_train,X_val,y_train,y_val = train_test_split(X[feat_cols],y,test_size=0.25,random_state = 1994)

# m=LGBMClassifier(n_estimators=5000,random_state=1994,learning_rate=0.05,num_leaves=131,
#                  objective='multiclass',num_class=7,feature_fraction=0.9,bagging_fraction=0.9,
# #                 reg_alpha=1
#                 )

# m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val, y_val)], early_stopping_rounds=50,verbose=200)
# p=m.predict(X_val)
# print(f1_score(y_val,p,average='macro'))

# y_pred1 = m.predict_proba(Xtest[feat_cols],num_iteration=m.best_iteration_)

In [ ]:
# X_train,X_val,y_train,y_val = train_test_split(X[feat_cols],y,test_size=0.3,random_state = 2021)
# m=LGBMClassifier(n_estimators=5000,random_state=1994,learning_rate=0.05,num_leaves=31,class_weight ='balanced', 
#                  objective='multiclass',num_class=7,feature_fraction=0.8,bagging_fraction=0.8
# #                 reg_alpha=1
#                 )

# m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val, y_val)], early_stopping_rounds=50,verbose=200)
# p=m.predict(X_val)
# print(f1_score(y_val,p,average='macro'))

# y_pred2 = m.predict_proba(Xtest[feat_cols],num_iteration=m.best_iteration_)

In [ ]:
# y_pred = np.argmax((y_pred1*0.5+y_pred2*0.5),1)


In [ ]:



# y_pred = m.predict(Xtest[feat_cols],num_iteration=m.best_iteration_)

# dftest['Top-up Month']=y_pred
# dftest['Top-up Month']=dftest['Top-up Month'].map(inverted_label_map)
# dftest[['ID','Top-up Month']]

# dftest['Top-up Month'].value_counts()


In [ ]:
dftest[['ID','Top-up Month']].to_csv('AV_frk6_v0.10.csv',index=False)

In [ ]:
# cat

In [ ]:
# 1-df['Top-up Month'].value_counts()/106677

In [ ]:
# from sklearn.utils import class_weight
# cw = list(class_weight.compute_class_weight('balanced',
#                                              np.unique(dftrain['Top-up Month']),
#                                              dftrain['Top-up Month']))
# cw

In [ ]:
# Xc,yc=dftrain.drop(['Top-up Month','ID'],axis=1),dftrain['Top-up Month']
# Xc['ManufacturerID']=Xc['ManufacturerID'].astype(str)
# Xc['ManufacturerID']=Xc['ManufacturerID'].astype('category')

# Xc['ZiPCODE']=Xc['ZiPCODE'].astype(str)
# Xc['ZiPCODE']=Xc['ZiPCODE'].astype('category')


# Xc['AGE']=Xc['AGE'].astype(str)
# Xc['AGE']=Xc['AGE'].astype('category')
# for x in cat:
#     Xc[x] = Xc[x].cat.add_categories("Others").fillna("Others")
    


    
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score,f1_score
# X_train,X_val,y_train,y_val = train_test_split(Xc,yc,test_size=0.25,random_state = 1994)



# categorical_features_indices = np.where(X_train.dtypes =='category')[0]
# categorical_features_indices

# from catboost import CatBoostClassifier
# m2=CatBoostClassifier(n_estimators=100,random_state=1994,eval_metric='TotalF1',learning_rate=0.1)
# m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val, y_val)], early_stopping_rounds=50,verbose=50,cat_features=categorical_features_indices)
# p=m2.predict(X_val)
# print(f1_score(y_val,p,average='macro'))

In [ ]:

# Xtest = dftest.drop(['Top-up Month','ID'],axis=1)

In [ ]:
# m2

In [ ]:
#lgb
# y_pred = m.predict(Xtest,num_iteration=m.best_iteration_)

#cb
# Xtest['ManufacturerID']=Xtest['ManufacturerID'].astype(str)
# Xtest['ManufacturerID']=Xtest['ManufacturerID'].astype('category')

# Xtest['ZiPCODE']=Xtest['ZiPCODE'].astype(str)
# Xtest['ZiPCODE']=Xtest['ZiPCODE'].astype('category')


# Xtest['AGE']=Xtest['AGE'].astype(str)
# Xtest['AGE']=Xtest['AGE'].astype('category')
# for x in cat:
#     Xtest[x] = Xtest[x].cat.add_categories("Others").fillna("Others")
# y_pred = m2.predict(Xtest,ntree_end=m2.best_iteration_)


In [ ]:
# y_pred

In [ ]:
# dftest['Top-up Month']=y_pred
# dftest['Top-up Month']=dftest['Top-up Month'].map(inverted_label_map)
# dftest[['ID','Top-up Month']]

# dftest['Top-up Month'].value_counts()

In [ ]:
# dftest[['ID','Top-up Month']].to_csv('AV_ltfs_sub_cb_6.csv',index=False)

In [ ]:
feat_cols

In [ ]:

Xtestt = dftest.drop(['Top-up Month','ID'],axis=1)

In [ ]:
err1=[]
y_pred_tot1=[]
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,GroupShuffleSplit,RepeatedStratifiedKFold
fold=RepeatedStratifiedKFold(n_splits=7, random_state=0,n_repeats=3)
for train_index, test_index in fold.split(X,y):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

#     m=LGBMClassifier(n_estimators=5000,random_state=1994,learning_rate=0.05,max_depth=-1,
#                  objective='multiclass',num_class=7,feature_fraction=0.9,bagging_fraction=0.9,reg_alpha=20,reg_lambda=10)
    cw={0:8,
     1:3,
     2:18,
     3:8,
     4:11,
     5:8,
     6:9}

#     m = CatBoostClassifier(cat_features=cat_feats,task_type="GPU",n_estimators=1000,
#                            learning_rate=0.07,class_weights=cw)
#     m.fit(X_train,y_train,verbose=300)#,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=50,verbose=200,eval_metric= evaluate_macroF1_lgb )
#     preds=m.predict(X_test)#,num_iteration=m.best_iteration_)
    m=CatBoostClassifier(n_estimators=1000,random_state=1994,eval_metric='TotalF1',learning_rate=0.05,
                      cat_features=cat_feats,task_type="GPU",class_weights=cw,random_strength=200,bagging_temperature=0.5,l2_leaf_reg=1)
    m.fit(X_train,y_train,eval_set=[(X_val, y_val)], early_stopping_rounds=50,verbose=200)
    preds=m.predict(X_test)#,num_iteration=m.best_iteration_)
    preds = [item for sublist in preds for item in sublist]
    print(set(preds))
    print("err: ",f1_score(y_test,preds,average='macro'))
    err1.append(f1_score(y_test,preds,average='macro'))
    p = m.predict_proba(Xtestt)#,num_iteration=m.best_iteration_)
    y_pred_tot1.append(p)

In [ ]:
err2=[]
y_pred_tot2=[]
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,GroupShuffleSplit,RepeatedStratifiedKFold
fold=RepeatedStratifiedKFold(n_splits=5, random_state=1994,n_repeats=2)
for train_index, test_index in fold.split(X,y):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=LGBMClassifier(n_estimators=5000,random_state=1994,learning_rate=0.05,max_depth=-1,
                 objective='multiclass',num_class=7,feature_fraction=0.9,bagging_fraction=0.9,reg_alpha=20,reg_lambda=10)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=50,verbose=200,eval_metric= evaluate_macroF1_lgb)
    preds=m.predict(X_test,num_iteration=m.best_iteration_)
    print(set(preds))
    print("err: ",f1_score(y_test,preds,average='macro'))
    err2.append(f1_score(y_test,preds,average='macro'))
    p = m.predict_proba(Xtestt,num_iteration=m.best_iteration_)
    y_pred_tot2.append(p)

In [ ]:
np.mean(err1),np.mean(err2)

In [ ]:
vv = np.argmax(np.mean(y_pred_tot1,0)*0.7+np.mean(y_pred_tot2,0)*0.3,1)
# vv

In [ ]:
# set(vv)

In [ ]:
dftest['Top-up Month']=vv
dftest['Top-up Month']=dftest['Top-up Month'].map(inverted_label_map)
dftest[['ID','Top-up Month']]

dftest['Top-up Month'].value_counts()

In [ ]:
dftest[['ID','Top-up Month']].to_csv('AV_frk6_stack-kf_v0.10.csv',index=False)

In [ ]:
dftest['Top-up Month']=np.argmax(np.mean(y_pred_tot2,0),1)
dftest['Top-up Month']=dftest['Top-up Month'].map(inverted_label_map)
dftest[['ID','Top-up Month']]

print(dftest['Top-up Month'].value_counts())
dftest[['ID','Top-up Month']].to_csv('AV_frk6_lgb-kf_v0.10.csv',index=False)



In [ ]:
dftest['Top-up Month']=np.argmax(np.mean(y_pred_tot1,0),1)
dftest['Top-up Month']=dftest['Top-up Month'].map(inverted_label_map)
dftest[['ID','Top-up Month']]

print(dftest['Top-up Month'].value_counts())
dftest[['ID','Top-up Month']].to_csv('AV_frk6_cb-kf_v0.10.csv',index=False)


